In [1]:
%%writefile mapper-tf.py
from __future__ import print_function
import sys
import re
import string

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

path = 'stop_words_en.txt'
tagWord = 'wordN'
tagWords = 'totalWordsNt'

stopWords = set()

with open(path) as stopWordsFile:
    for line in stopWordsFile:
        try:         
            stopWords.add(unicode(line.strip().lower()))
        except ValueError as e:
            continue

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        print("reporter:counter:Wiki stats,Wrong articles found,%d" % 1, file=sys.stderr)
        continue

    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    
    wordsInArticle = 0
    
    for word in words:
        word = word.strip(string.punctuation).lower()
        if word == "":
            print("reporter:counter:Wiki stats,Null words found,%d" % 1, file=sys.stderr)
            continue
        if word in stopWords:
            print("reporter:counter:Wiki stats,Stop words found,%d" % 1, file=sys.stderr)
            continue
        wordsInArticle += 1
        print("reporter:counter:Wiki stats,Total words found,%d" % 1, file=sys.stderr)
        print(article_id, tagWord, word, 1, sep='\t')

    print(article_id, tagWords, "-", wordsInArticle, sep='\t')


Overwriting mapper-tf.py


In [2]:
%%writefile reducer-tf.py
import sys


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

current_article_id = None
current_word = None
word_sum, wordsInArticle = 0, 0

tagWord = 'wordN'
tagWords = 'totalWordsNt'
tagTf = 'tf'

for line in sys.stdin:
    try:
        article_id, tag, word, count = unicode(line.strip()).split('\t')
        #print "after parse:", article_id, word, count
        if article_id == "" or tag == "" or word == "" or count == "":   # sanity check
            continue
        count = int(count)
    except ValueError as e:
        #print "Parse error:", e
        continue

    if current_article_id != article_id:
        if tag == tagWords: # get total words in article
            wordsInArticle = count
            continue
        if current_article_id != None:
            print "%s\t%s\t%s\t%f" % (current_word, tagTf, current_article_id, 1.0*word_sum/wordsInArticle)
            word_sum = 0
            current_word = word
        current_article_id = article_id
        #print "wordsInArticle:", word, wordsInArticle
        
    if current_word != word:
        if current_word != None:
            print "%s\t%s\t%s\t%f" % (current_word, tagTf, current_article_id, 1.0*word_sum/wordsInArticle)
            #print "%s\t%s\t%d\t%d" % (current_article_id, current_word, word_sum, wordsInArticle)
        word_sum = 0
        current_word = word
    word_sum += count

if current_article_id:
    print "%s\t%s\t%s\t%f" % (current_word, tagTf, current_article_id, 1.0*word_sum/wordsInArticle)


Overwriting reducer-tf.py


In [3]:
%%writefile mapper-idf.py
from __future__ import print_function
import sys

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

for line in sys.stdin:
    word, _, _, _ = unicode(line.strip()).split('\t')
    print("reporter:counter:Idf stats,Total words found,%d" % 1, file=sys.stderr)
    print(word, 1, sep='\t')


Overwriting mapper-idf.py


In [4]:
%%writefile reducer-idf.py
import sys
from math import log

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

current_key = None
word_sum = 0
tag = "idf"

for line in sys.stdin:
    try:
        key, count = unicode(line.strip()).split('\t', 1)
        if key == "" or count == "":    # sanity check
            continue
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%s\t%f" % (current_key, tag, (1.0/log(1.0 + word_sum)))
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%s\t%f" % (current_key, tag, (1.0/log(1.0 + word_sum)))

Overwriting reducer-idf.py


In [5]:
%%writefile mapper-tf-idf.py
from __future__ import print_function
import sys


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

tf = "tf"
idf = "idf"

for line in sys.stdin:
    inVal = unicode(line.strip()).split('\t')
    if len(inVal) > 4:
        print("reporter:counter:Wiki stats,tf idf > 4 parsing errors,%d" % 1, file=sys.stderr)
        continue
    try:
        if inVal[1] == idf: # idf data
            print("reporter:counter:Wiki stats,idf words,%d" % 1, file=sys.stderr)
            print(inVal[0], idf, float(inVal[2]), sep='\t')
            
        elif inVal[1] == tf: # tf data
            print("reporter:counter:Wiki stats,tf words,%d" % 1, file=sys.stderr)
            print(inVal[0], tf, inVal[2], float(inVal[3]), sep='\t')
            
        else:
            print("reporter:counter:Wiki stats,tf idf last else parsing errors,%d" % 1, file=sys.stderr)
    except ValueError as e:
        print("reporter:counter:Wiki stats,tf idf to float errors,%d" % 1, file=sys.stderr)
        continue


Overwriting mapper-tf-idf.py


In [6]:
%%writefile reducer-tf-idf.py
import sys
from math import log

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

tf = "tf"
idf = "idf"

for line in sys.stdin:
    inVal = unicode(line.strip()).split('\t')
    if (len(inVal) < 2 and len(inVal) > 4):      # sanity check
        continue
    try:
        if inVal[1] == idf:
            idfWord, _, idfDt = inVal
            idfDt = float(idfDt)

        if inVal[1] == tf:
            tfWord, _, article_id, tfNt = inVal
            tfNt = float(tfNt)
            if idfWord == tfWord:
                print "%s\t%s\t%f" % (idfWord, article_id, tfNt*idfDt)

    except ValueError as e:
        continue


Overwriting reducer-tf-idf.py


In [7]:
%%bash

OUT_DIR_TF="tf_result"$(date +"%s%6N")

NUM_REDUCERS=3

hdfs dfs -rm -r -skipTrash ${OUT_DIR_TF} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D stream.num.map.output.key.fields=3 \
-D stream.num.reduce.output.fields=3 \
-D mapreduce.partition.keypartitioner.options=-k1,1 \
-D mapreduce.partition.keycomparator.options=-k1,3 \
-D mapreduce.job.name="Tf-Idf tf task" \
-D mapreduce.job.reduces=${NUM_REDUCERS} \
-files mapper-tf.py,reducer-tf.py,/datasets/stop_words_en.txt  \
-mapper "python mapper-tf.py" \
-reducer "python reducer-tf.py" \
-input /data/wiki/en_articles_part \
-partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
-output ${OUT_DIR_TF} > /dev/null

OUT_DIR_IDF="idf_result"$(date +"%s%6N")
NUM_REDUCERS=3

hdfs dfs -rm -r -skipTrash ${OUT_DIR_IDF} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Tf-Idf idf task" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper-idf.py,reducer-idf.py \
    -mapper "python mapper-idf.py" \
    -reducer "python reducer-idf.py" \
    -input ${OUT_DIR_TF} \
    -output ${OUT_DIR_IDF} > /dev/null


OUT_DIR_TF_IDF="tf-idf-out"$(date +"%s%6N")
NUM_REDUCERS=3

hdfs dfs -rm -r -skipTrash ${OUT_DIR_TF_IDF} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D stream.num.map.output.key.fields=2 \
-D stream.num.reduce.output.fields=2 \
-D mapreduce.partition.keypartitioner.options=-k1,1 \
-D mapreduce.partition.keycomparator.options=-k1,2 \
-D mapreduce.job.name="Tf-Idf tf*idf task" \
-D mapreduce.job.reduces=${NUM_REDUCERS} \
-files mapper-tf-idf.py,reducer-tf-idf.py \
-mapper "python mapper-tf-idf.py" \
-reducer "python reducer-tf-idf.py" \
-input ${OUT_DIR_TF},${OUT_DIR_IDF} \
-partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
-output ${OUT_DIR_TF_IDF} > /dev/null

hdfs dfs -text ${OUT_DIR_TF_IDF}/* | grep -E 'labor\W12\W'


labor	12	0.000351


rm: `tf_result1549615133620370': No such file or directory
19/02/08 08:38:57 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/08 08:38:57 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/08 08:38:58 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/08 08:38:58 INFO mapreduce.JobSubmitter: number of splits:2
19/02/08 08:38:58 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549545179555_0026
19/02/08 08:38:59 INFO impl.YarnClientImpl: Submitted application application_1549545179555_0026
19/02/08 08:38:59 INFO mapreduce.Job: The url to track the job: http://e5801941607d:8088/proxy/application_1549545179555_0026/
19/02/08 08:38:59 INFO mapreduce.Job: Running job: job_1549545179555_0026
19/02/08 08:39:06 INFO mapreduce.Job: Job job_1549545179555_0026 running in uber mode : false
19/02/08 08:39:06 INFO mapreduce.Job:  map 0% reduce 0%
19/02/08 08:39:23 INFO mapreduce.Job:  map 18% reduce 0%
19/02/08 08:39:29 INF